In [1]:
import pandas as pd
import numpy as np
import sklearn.naive_bayes as naive_bayes
import itertools


data = pd.read_csv('Purchase_Likelihood.csv')
data.head()

,group_size,homeowner,married_couple,A
0,2,0,1,1
1,2,0,1,1
2,2,0,1,1
3,2,0,1,1
4,2,0,1,1


In [2]:
# Define a function to visualize the percent of a particular target category by a nominal predictor
def RowWithColumn (
   rowVar,          # Row variable
   columnVar,       # Column predictor
   show = 'ROW'):   # Show ROW fraction, COLUMN fraction, or BOTH table

    countTable = pd.crosstab(index = rowVar, columns = columnVar, margins = False, dropna = True)
    print("Frequency Table: \n", countTable)
    print( )

#     if (show == 'ROW' or show == 'BOTH'):
#         rowFraction = countTable.div(countTable.sum(1), axis='index')
#         print("Row Fraction Table: \n", rowFraction)
#         print( )

#     if (show == 'COLUMN' or show == 'BOTH'):
#         columnFraction = countTable.div(countTable.sum(0), axis='columns')
#         print("Column Fraction Table: \n", columnFraction)
#         print( )

    return countTable


In [3]:
feature = ['group_size','homeowner','married_couple']
target = 'A'

df = []
data = data.dropna()
for pred in feature:
    df.append(RowWithColumn(rowVar = data[target], columnVar = data[pred], show = 'BOTH'))
    
data[feature] = 2 - data[feature]

xTrain = data[feature].astype('category')
yTrain = data[target].astype('category')

_objNB = naive_bayes.BernoulliNB(alpha = 0)
thisFit = _objNB.fit(xTrain, yTrain)

print('Probability of each class')
print(np.exp(thisFit.class_log_prior_))

print('Empirical probability of features given a class, P(x_i|y)')
print(np.exp(thisFit.feature_log_prob_))

print('Number of samples encountered for each class during fitting')
print(thisFit.class_count_)

print('Number of samples encountered for each (class, feature) during fitting')
print(thisFit.feature_count_)

# yTrain_predProb = _objNB.predict_proba(xTrain)

# # Create the all possible combinations of the features' values
# xTest = pd.DataFrame(list(itertools.product([0,1], repeat = len(feature))), columns = feature)

# # Score the xTest and append the predicted probabilities to the xTest
# yTest_predProb = pd.DataFrame(_objNB.predict_proba(xTest))
# yTest_score = pd.concat([xTest, yTest_predProb], axis = 1)

# print(yTest_predProb)
# print(yTest_score)

Frequency Table: 
 group_size       1      2     3    4
A                                   
0           115460  25728  2282  221
1           329552  91065  5069  381
2            74293  19600  1505   93

Frequency Table: 
 homeowner       0       1
A                        
0           78659   65032
1          183130  242937
2           46734   48757

Frequency Table: 
 married_couple       0      1
A                            
0               117110  26581
1               333272  92795
2                75310  20181

Probability of each class
[0.21599582 0.64046244 0.14354174]
Empirical probability of features given a class, P(x_i|y)
[[0.8035298  1.         1.        ]
 [0.77347459 1.         1.        ]
 [0.77801049 1.         1.        ]]
Number of samples encountered for each class during fitting
[143691. 426067.  95491.]
Number of samples encountered for each (class, feature) during fitting
[[115460. 143691. 143691.]
 [329552. 426067. 426067.]
 [ 74293.  95491.  95491.]]


C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [4]:
df[0]

group_size,1,2,3,4
A,,,,
0,115460,25728,2282,221
1,329552,91065,5069,381
2,74293,19600,1505,93


In [5]:
df[1]

homeowner,0,1
A,,
0,78659,65032
1,183130,242937
2,46734,48757


In [13]:
df[2]

married_couple,0,1
A,,
0,117110,26581
1,333272,92795
2,75310,20181


In [8]:
# Define a function that performs the Chi-square test
import scipy
def cramerVFunc (
    xCat,           # input categorical feature
    yCat,           # input categorical target variable   # debugging flag (Y/N) 
    debug = 'Y'     
):

    obsCount = pd.crosstab(index = xCat, columns = yCat, margins = False, dropna = True)
    cTotal = obsCount.sum(axis = 1)
    rTotal = obsCount.sum(axis = 0)
    nTotal = np.sum(rTotal)
    expCount = np.outer(cTotal, (rTotal / nTotal))
    
    if (debug == 'Y'):
        print('Observed Count:\n', obsCount)
        print('Column Total:\n', cTotal)
        print('Row Total:\n', rTotal)
        print('Overall Total:\n', nTotal)
        print('Expected Count:\n', expCount)
        print('\n')

       
    chiSqStat = ((obsCount - expCount)**2 / expCount).to_numpy().sum()

    cramerV = chiSqStat / nTotal
    if (cTotal.size > rTotal.size):
        cramerV = cramerV / (rTotal.size - 1.0)
    else:
        cramerV = cramerV / (cTotal.size - 1.0)
    cramerV = np.sqrt(cramerV)

    return (cramerV)


In [12]:
for i in list(df[0].columns.values):
    print('For Feature {} '.format(i),cramerVFunc(df[0][i],df[0].index.values))
print()
for i in list(df[1].columns.values):
    print('For Feature {} '.format(i),cramerVFunc(df[1][i],df[1].index.values))
print()    
for i in list(df[2].columns.values):
    print('For Feature {} '.format(i),cramerVFunc(df[2][i],df[2].index.values))

Observed Count:
 1      74293   115460  329552
col_0                        
0           0       1       0
1           0       0       1
2           1       0       0
Column Total:
 col_0
0    1
1    1
2    1
dtype: int64
Row Total:
 1
74293     1
115460    1
329552    1
dtype: int64
Overall Total:
 3
Expected Count:
 [[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]]


For Feature 1  1.0
Observed Count:
 col_0  0  1  2
2             
19600  0  0  1
25728  1  0  0
91065  0  1  0
Column Total:
 2
19600    1
25728    1
91065    1
dtype: int64
Row Total:
 col_0
0    1
1    1
2    1
dtype: int64
Overall Total:
 3
Expected Count:
 [[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]]


For Feature 2  1.0
Observed Count:
 col_0  0  1  2
3             
1505   0  0  1
2282   1  0  0
5069   0  1  0
Column Total:
 3
1505    1
2282    1
5069    1
dtype: int64
Row Total:
 col_0
0    1
1